In [ ]:
import numpy as np
import cvxpy as cp
import scipy as sc
import pennylane as qml
import torch
import torch.nn as nn
import torch.nn.functional as Fn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
import torch
import torch.nn as nn

class neural_function(nn.Module):
    def __init__(self, dimension, hidden_layers):
        super(neural_function, self).__init__()

        self.dimension = dimension
        self.hidden_layers = hidden_layers

        # Create a list to hold the hidden layer modules
        self.hidden_layer_modules = nn.ModuleList()

        # Add the input layer
        self.hidden_layer_modules.append(nn.Linear(dimension, hidden_layers[0]))

        # Add the hidden layers
        for i in range(1, len(hidden_layers)):
            self.hidden_layer_modules.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))

        # Add the output layer
        self.lin_end = nn.Linear(hidden_layers[-1], 1)

    def forward(self, input):
        y = input.float()

        # Forward pass through each hidden layer
        for layer in self.hidden_layer_modules:
            y = torch.sigmoid(layer(y))
        # Forward pass through the output layer
        y = self.lin_end(y)

        return y

In [ ]:
import numpy as np

def generate_diagonal_positive_definite_matrices(dim):
    while True:
        # Generate random positive values for the diagonal entries
        diagonal_values_1 = np.random.rand(dim)
        diagonal_values_2 = np.random.rand(dim)

        # Construct the diagonal matrices A and B
        A = np.diag(diagonal_values_1)
        B = np.diag(diagonal_values_2)

        # Compute the trace of A
        trace_A = np.trace(A)
        trace_B = np.trace(B)

        # Normalize the matrices to have trace 1
        A /= trace_A
        B /= trace_B

        # Check if matrices A and B commute
        commutation_check = np.dot(A, B) - np.dot(B, A)

        if np.allclose(commutation_check, np.zeros((dim, dim))):
            return A, B

# Generate diagonal positive definite matrices A and B with trace 1 and they commute
N = 4
dim = N # Dimension of the matrices
A, B = generate_diagonal_positive_definite_matrices(dim)

# Print matrices A and B
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

In [ ]:
# quantum circuit settings
num_wires = 4
num_layers = 10
num_shots = 1
num_of_samples = 100

# initiate the quantum device
device = qml.device("default.mixed", wires=num_wires, shots=num_of_samples)

@qml.qnode(device)
def measure_rho(param):
    qml.QubitDensityMatrix(A, wires=[0, 1])
    qml.RandomLayers(param, wires=[0, 1])
    # measure in the computational basis
    result = qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
    return result

@qml.qnode(device)
def measure_sigma(param):
    qml.QubitDensityMatrix(sigma, wires=[0, 1])
    qml.RandomLayers(param, wires=[0, 1])
    # measure in the computational basis
    result = qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
    return result

In [ ]:
# system size
N = 4

# ignore the division by zero
np.seterr(divide = 'ignore')

# quantum relative entropy
H_rho = np.real(np.trace(rho_test @ (sc.linalg.logm(rho_test) - sc.linalg.logm(sigma_test))))

np.seterr(divide = 'warn')

# print the result
print(H_rho)

In [ ]:
#@title Optimization using Gradient Descent (without neural network)

# parameters of the optimization
num_of_epochs = 300
learning_rate = 0.1
num_of_samples = 100
deviation = 1

# initialize the neural network and quantum circuit parameters
W = deviation*np.random.rand(3,3)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))

# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):

  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # parameter-shift for the first term

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum_1 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                forward_sum_1 += W[result[0][sample]][result[1][sample]]

      # normalize the forward sum
            forward_sum_1 = forward_sum_1/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # backward evaluation
            backward_sum_1 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                backward_sum_1 += W[result[0][sample]][result[1][sample]]

      # normalize the backward sum
            backward_sum_1 = backward_sum_1/num_of_samples

      # parameter-shift for the second term

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum_2 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                forward_sum_2 += W[result[0][sample]][result[1][sample]]

      # normalize the forward sum
            forward_sum_2 = forward_sum_2/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # backward evaluation
            backward_sum_2 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                backward_sum_2 += W[result[0][sample]][result[1][sample]]

      # normalize the backward sum
            backward_sum_2 = backward_sum_2/num_of_samples

      # parameter-shift rule
            gradients[i, j] =  0.5 * (forward_sum_1 - backward_sum_1) - 0.5 * (forward_sum_2 - backward_sum_2)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init += learning_rate*gradients

  # evaluate the gradient with respect to the eigenvalues

  # 1 , 1
    E = np.zeros_like(W)
    E[1][1] = 1
    dW1_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW1_first_term += E[result[0][sample]][result[1][sample]]


    # normalize it
    dW1_first_term = dW1_first_term/num_of_samples

    dW1_sec_term = 0
    result = measure_sigma(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW1_sec_term += E[result[0][sample]][result[1][sample]]*np.exp(W[result[0][sample]][result[1][sample]])

    # normalize it
    dW1_sec_term = dW1_sec_term/num_of_samples

    # 1 , -1
    E = np.zeros_like(W)
    E[1][-1] = 1
    dW2_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW2_first_term += E[result[0][sample]][result[1][sample]]

    # normalize it
    dW2_first_term = dW2_first_term/num_of_samples

    dW2_sec_term = 0
    result = measure_sigma(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW2_sec_term += E[result[0][sample]][result[1][sample]]*np.exp(W[result[0][sample]][result[1][sample]])

    # normalize it
    dW2_sec_term = dW2_sec_term/num_of_samples

    # -1 , 1
    E = np.zeros_like(W)
    E[-1][1] = 1
    dW3_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW3_first_term += E[result[0][sample]][result[1][sample]]

    # normalize it
    dW3_first_term = dW3_first_term/num_of_samples

    dW3_sec_term = 0
    result = measure_sigma(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW3_sec_term += E[result[0][sample]][result[1][sample]]*np.exp(W[result[0][sample]][result[1][sample]])

    # normalize it
    dW3_sec_term = dW3_sec_term/num_of_samples

    # -1 , -1
    E = np.zeros_like(W)
    E[-1][-1] = 1
    dW4_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW4_first_term += E[result[0][sample]][result[1][sample]]

    # normalize it
    dW4_first_term = dW4_first_term/num_of_samples

    dW4_sec_term = 0
    result = measure_sigma(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW4_sec_term += E[result[0][sample]][result[1][sample]]*np.exp(W[result[0][sample]][result[1][sample]])

    # normalize it
    dW4_sec_term = dW4_sec_term/num_of_samples

    # update the eigenvalues
    W[1][1] += learning_rate*(dW1_first_term - dW1_sec_term)
    W[1][-1] += learning_rate*(dW2_first_term - dW2_sec_term)
    W[-1][1] += learning_rate*(dW3_first_term - dW3_sec_term)
    W[-1][-1] += learning_rate*(dW4_first_term - dW4_sec_term)

    # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        first_term += W[result[0][sample]][result[1][sample]]

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    result = measure_sigma(param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        second_term += np.exp(W[result[0][sample]][result[1][sample]])

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append(1 + first_term - second_term)

  # print the cost
    print(1 + first_term - second_term)

In [ ]:
#@title Optimization using Gradient Descent

# parameters of the optimization
num_of_epochs = 1000
learning_rate = 0.08
num_of_samples = 200
dimension = 2
hidden_layer = 5
deviation = -1

# initialize the neural network and quantum circuit parameters
W = deviation*np.random.rand(3,3)
np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))

# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):

  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                forward_sum += W[result[0][sample]][result[1][sample]]

      # normalize the forward sum
            forward_sum = forward_sum/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # backward evaluation
            backward_sum = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                backward_sum += W[result[0][sample]][result[1][sample]]

      # normalize the backward sum
            backward_sum = backward_sum/num_of_samples
      #print(backward_sum)

      # parameter-shift rule
            gradients[i, j] = - 0.5 * (forward_sum - backward_sum)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init -= learning_rate*gradients
  #print(gradients)

  # evaluate the gradient with respect to the NN parameters


  # evaluate the gradient with respect to the eigenvalues

  # 1 , 1
    E = np.zeros_like(W)
    E[1][1] = 1
    dW1 = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW1 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW1 = dW1/num_of_samples
    dW1 += (np.exp(W[1][1])/N)

    # 1 , -1
    E = np.zeros_like(W)
    E[1][-1] = 1
    dW2 = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW2 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW2 = dW2/num_of_samples
    dW2 += (np.exp(W[1][-1])/N)

    # -1 , 1
    E = np.zeros_like(W)
    E[-1][1] = 1
    dW3 = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW3 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW3 = dW3/num_of_samples
    dW3 += (np.exp(W[-1][1])/N)

    # -1 , -1
    E = np.zeros_like(W)
    E[-1][-1] = 1
    dW4 = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW4 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW4 = dW4/num_of_samples
    dW4 += (np.exp(W[-1][-1])/N)


    # update the eigenvalues
    W[1][1] -= learning_rate*dW1
    W[1][-1] -= learning_rate*dW2
    W[-1][1] -= learning_rate*dW3
    W[-1][-1] -= learning_rate*dW4

    # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        first_term += W[result[0][sample]][result[1][sample]]

    # normalize the cost sum
    first_term = first_term/num_of_samples

    # # Second term evaluation
    second_term = np.exp(W[1][1]) + np.exp(W[1][-1]) + np.exp(W[-1][1]) + np.exp(W[-1][-1])

    # normalize the second term sum
    second_term = second_term/N

    # add the cost function to the store
    cost_func_store.append(first_term - second_term + 1)

    # print the cost
    print(first_term - second_term + 1)

In [ ]:
class neural_function(nn.Module):
    def __init__(self,dimension,hidden_layer):
        super(neural_function, self).__init__()

        self.dimension = dimension
        self.hidden_layer = hidden_layer
        self.lin1 = nn.Linear(self.dimension, self.hidden_layer)
        self.lin_end = nn.Linear(self.hidden_layer, 1)

    def forward(self, input):
        # y = Fn.leaky_relu(self.lin1(input))
        y = torch.sigmoid(self.lin1(input.float()))
        y = self.lin_end(y)

        return y

In [ ]:
#@title Optimization using Gradient Descent

# parameters of the optimization
num_of_epochs = 1000
learning_rate = 0.05
num_of_samples = 100
dimension = 2
hidden_layer = 10

# initialize the neural network and quantum circuit parameters
neural_fn = neural_function(dimension, hidden_layer)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))

# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):

  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[0][sample], result[1][sample]])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum += nn_result[0].detach().numpy()

      # normalize the forward sum
            forward_sum = forward_sum/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # backward evaluation
            backward_sum = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[0][sample], result[1][sample]])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum += nn_result[0].detach().numpy()

      # normalize the backward sum
            backward_sum = backward_sum/num_of_samples
      #print(backward_sum)

      # parameter-shift rule
            gradients[i, j] = - 0.5 * (forward_sum - backward_sum)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init -= learning_rate*gradients
  #print(gradients)

  # evaluate the gradient with respect to the NN parameters


  # evaluate the first term
    grad_w1 = torch.zeros_like(neural_fn.lin1.weight)
    grad_b1 = torch.zeros_like(neural_fn.lin1.bias)
    grad_w2 = torch.zeros_like(neural_fn.lin_end.weight)
    grad_b2 = torch.zeros_like(neural_fn.lin_end.bias)

    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[0][sample], result[1][sample]])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        nn_result.backward()
        grad_w1 += neural_fn.lin1.weight.grad*(1/num_of_samples)
        grad_b1 += neural_fn.lin1.bias.grad*(1/num_of_samples)
        grad_w2 += neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2 += neural_fn.lin_end.bias.grad*(1/num_of_samples)
        neural_fn.lin1.weight.grad.zero_()
        neural_fn.lin1.bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()

  # evaluate the second term
    grad_w1_2 = torch.zeros_like(neural_fn.lin1.weight.grad)
    grad_b1_2 = torch.zeros_like(neural_fn.lin1.bias.grad)
    grad_w2_2 = torch.zeros_like(neural_fn.lin_end.weight.grad)
    grad_b2_2 = torch.zeros_like(neural_fn.lin_end.bias.grad)

    for sample in range(num_of_samples):
        result = np.random.choice([-1, 1], size=2)
        nn_result = neural_fn(torch.from_numpy(result.flatten()))
        nn_result.backward()
        grad_w1_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin1.weight.grad*(1/num_of_samples)
        grad_b1_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin1.bias.grad*(1/num_of_samples)
        grad_w2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        neural_fn.lin1.weight.grad.zero_()
        neural_fn.lin1.bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()

  # evaluate the difference, i.e., the gradient

    nn_grad_W1 = grad_w1_2 - grad_w1
    nn_grad_b1 = grad_b1_2 - grad_b1
    nn_grad_W2 = grad_w2_2 - grad_w2
    nn_grad_b2 = grad_b2_2 - grad_b2

  # update the NN weights and normalize them
    with torch.no_grad():
        neural_fn.lin1.weight -= learning_rate*nn_grad_W1
        neural_fn.lin1.bias -= learning_rate*nn_grad_b1
        neural_fn.lin_end.weight -= learning_rate*nn_grad_W2
        neural_fn.lin_end.bias -= learning_rate*nn_grad_b2

  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[0][sample], result[1][sample]])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        first_term += nn_result[0].detach().numpy()

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    for sample in range(num_of_samples):
        result = np.random.choice([-1, 1], size=2)
        nn_result = neural_fn(torch.from_numpy(result.flatten()))
        second_term += np.exp(nn_result[0].detach().numpy())

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append(first_term - second_term + 1)

  # print the cost
    print(first_term - second_term + 1)